In [1]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
import pandas as pd
from tqdm import tqdm
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertTokenizer, BertModel
import numpy as np
from sentence_transformers import SentenceTransformer,util
import matplotlib.pyplot as plt
import csv
from numpy.linalg import norm
from numpy import dot

/opt/anaconda3/envs/mysite/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device="cuda" if torch.cuda.is_available() else "cpu"

In [3]:
intent_to_num={"학식":0,"편의시설":1,"연락처":2,"학사일정":3,"공지사항":4,"학교 날씨":5,"학교 정보":6,"졸업 요건":7,"열람실":8}
num_to_intent={value:key for key,value in intent_to_num.items()}
print(num_to_intent)

{0: '학식', 1: '편의시설', 2: '연락처', 3: '학사일정', 4: '공지사항', 5: '학교 날씨', 6: '학교 정보', 7: '졸업 요건', 8: '열람실'}


In [4]:
df = pd.read_excel("dataset/의도분류질문.xlsx")
print(df.head())

   의도          질문
0  학식  오늘 학식을 알려줘
1  학식  오늘 아침을 알려줘
2  학식  오늘 점심을 알려줘
3  학식  오늘 저녁을 알려줘
4  학식   오늘 학식 뭐나와


In [109]:
# model = SentenceTransformer("snunlp/KR-SBERT-V40K-klueNLI-augSTS")

In [7]:
sentences=[df.iloc[i,1] for i in range(len(df))]
sentences_class=[intent_to_num[df.iloc[i,0]] for i in range(len(df))]
print(sentences_class)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 4, 4, 4, 4, 4, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 8, 8, 8]


In [11]:
import torch
import numpy as np
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel

# KoBERT 모델 및 토크나이저 로드
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
model = BertModel.from_pretrained('skt/kobert-base-v1')

def embed_sentence(sentence, keyword_weights):
    inputs = tokenizer(sentence, return_tensors='pt', truncation=True, padding=True)
    outputs = model(**inputs)
    token_embeddings = outputs.last_hidden_state.squeeze(0)
    
    token_ids = inputs['input_ids'].squeeze()
    tokens = tokenizer.convert_ids_to_tokens(token_ids)
    token_weights = np.array([keyword_weights.get(token, 1.0) for token in tokens])
    
    weighted_embeddings = token_embeddings * torch.tensor(token_weights).unsqueeze(1).to(token_embeddings.device)
    return weighted_embeddings.detach().cpu()

def pad_sequences(sequences, maxlen=None, dtype=torch.float32, padding_value=0.0):
    if maxlen is None:
        maxlen = max(seq.size(0) for seq in sequences)
    padded_sequences = []
    for seq in sequences:
        if seq.size(0) < maxlen:
            padding = torch.full((maxlen - seq.size(0), seq.size(1)), padding_value, dtype=dtype)
            padded_seq = torch.cat([seq, padding], dim=0)
        else:
            padded_seq = seq
        padded_sequences.append(padded_seq)
    return torch.stack(padded_sequences)

def cosine_similarity(vec1, vec2):
    flattened_vec1 = vec1.flatten()
    flattened_vec2 = vec2.flatten()
    return np.dot(flattened_vec1, flattened_vec2) / (np.linalg.norm(flattened_vec1) * np.linalg.norm(flattened_vec2))

keyword_weights = {
    "총장": 1.2,
    "학식": 1.2, 
}

embedding_vectors = [embed_sentence(sentence, keyword_weights) for sentence in sentences]
embedding_vectors = pad_sequences(embedding_vectors)
print(embedding_vectors.shape)

similarity = cosine_similarity(embedding_vectors[1], embedding_vectors[2])
print(f"Cosine Similarity: {similarity}")


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


torch.Size([166, 23, 768])
Cosine Similarity: 0.803135922068623


In [112]:
# embedding_vectors=[model.encode(sentence) for sentence in sentences]
# embedding_vectors=torch.tensor(embedding_vectors)
# print(embedding_vectors.shape)

AttributeError: 'BertModel' object has no attribute 'encode'

In [ ]:
def cos_sim(A, B):
    return dot(A, B) / (norm(A) * norm(B))

In [12]:
def predict(sentence):
    # print(f"원래 문장 : {sentence}")
    encoded_sentence=embed_sentence(sentence, keyword_weights)
    embedding_vectors = pad_sequences([encoded_sentence])
    encoded_sentence=torch.tensor(encoded_sentence)
    cos_sim=cosine_similarity(encoded_sentence,embedding_vectors)
    # print(f"가장 높은 코사인 유사도 idx: {int(np.argmax(cos_sim))}")

    best_sim_idx=int(np.argmax(cos_sim))
    selected_question=sentences[best_sim_idx]
    # print(f"선택된 질문 : {selected_question}")
    # print(f"선태괸 질문과의 유사도 : {float(cos_sim.max())}")
    # print(f"선택된 질문의 class : {num_to_intent[sentences_class[best_sim_idx]]}")
    
    return sentences_class[best_sim_idx], selected_question,float(cos_sim.max())
    # return int(np.argmax(cos_sim))

s = "학교 교가는 뭐야?"
predict(s)

/var/folders/6t/w80zhm5565j25gfph102_yfm0000gn/T/ipykernel_18559/945312013.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  encoded_sentence=torch.tensor(encoded_sentence)


(0, '오늘 학식을 알려줘', 1.0)

In [120]:
s = "전화번호가 궁금해?"

import time

start_time=time.time()

predict(s)

end_time=time.time()



/var/folders/6t/w80zhm5565j25gfph102_yfm0000gn/T/ipykernel_38451/1184705647.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  encoded_sentence=torch.tensor(encoded_sentence)


In [121]:
data=pd.read_csv("dataset/my_data_0702.csv")

In [122]:
f=open("failure.txt",'w')
correct=0
min_cos_sim=1000
max_cos_sim=-1000
for i in tqdm(range(len(data))):
    label=data.iloc[i]['label']-2 if data.iloc[i]['label'] >=8 else data.iloc[i]['label'] -1
    predict_class, pred_str,best_cos_sim=predict(data.iloc[i]['sentence'])

    if predict_class==label:
        min_cos_sim=min(min_cos_sim,best_cos_sim)
        correct+=1
    else:
        max_cos_sim=max(max_cos_sim,best_cos_sim)
        f.write(f"원래 문장 : {data.iloc[i]['sentence']}, 원래 라벨 : {label}, 예측 라벨 : {predict_class}\n")
        f.write(f"예측한 문장 : {pred_str}, 코사인 유사도 : {best_cos_sim}\n")

  0%|          | 0/4487 [00:00<?, ?it/s]/var/folders/6t/w80zhm5565j25gfph102_yfm0000gn/T/ipykernel_38451/1184705647.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  encoded_sentence=torch.tensor(encoded_sentence)
100%|██████████| 4487/4487 [03:16<00:00, 22.85it/s]


In [123]:
print(f"예측 정확도 : {correct/len(data)*100} % ")
print(f"최저 유사도 : {min_cos_sim}")
print(f"최고 유사도 : {max_cos_sim}")

예측 정확도 : 11.23244929797192 % 
최저 유사도 : 0.9999999999999998
최고 유사도 : 1.0000000000000002
